In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('EV_files/EV_train.csv')
labels = pd.read_csv('EV_files/EV_train_labels.csv')

df.set_index('House ID', inplace=True)
labels.set_index('House ID', inplace=True)

houses_w_nans = df[pd.isnull(df).any(axis=1)]
houses_missing_data = houses_w_nans.index.tolist()
houses_missing_data

df = df[~df.index.isin(houses_missing_data)]
labels = labels[~labels.index.isin(houses_missing_data)]

ev_label = labels.max(axis=1).rename('Has_EV')
df = pd.concat((df, ev_label), axis=1, join='inner')

In [2]:
y = df.pop('Has_EV').as_matrix()
X = df.as_matrix()
# reshape X for LSTM as(samples, time steps, features)
X = np.reshape(X, X.shape + (1,))
X.shape

(1586, 2880, 1)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [4]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
numpy.random.seed(15)  # for reproducibility

Using Theano backend.


In [10]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=X_train.shape[1:]))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64, class_weight=class_weight)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________
None
Train on 1268 samples, validate on 318 samples
Epoch 1/1
1268/1268 [==============================] - 684s - loss: 0.6288 - acc: 0.6940 - val_loss: 0.6177 - val_acc: 0.6950


In [12]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 69.50%


In [ ]:
model.predict_proba(X_test)